In [537]:
import copy
import sys
import numpy as np

sys.path.append('../') #server per fare l'import di file in un'altra directory sullo stesso "livello"
from src.NeuralNetwork import Layer, NeuralNetwork
from src.function import IdentityFunction

In [538]:
def calculate_total_loss(target_output: np.matrix, output_nn: np.matrix) -> np.float64:
    if target_output is None:
        raise ValueError("target_output must be != None")
    if output_nn is None:
        raise ValueError("output_nn must be != None")
    if target_output.shape != output_nn.shape:
        raise ValueError(f"target_output ({target_output.shape}) and output_nn ({output_nn.shape}) must have the same shape")

    error_vector = np.sum(np.square(target_output - output_nn), axis=1) * 0.5
    error_total = np.sum(error_vector)
    return error_total

In [539]:
def gradient_check(neuralNetwork1, neuralNetwork2, target_inputs, target_outputs, epsilon=1e-7):

    is_correct = True

    # Applicazione backpropagation per il nostro algoritmo
    for target_input, target_output in zip(target_inputs, target_outputs):
        neuralNetwork2._backpropagation(target_input=target_input, target_output=target_output)

    # Gradiant checking per ogni singolo peso
    for layer in range(len(neuralNetwork1.layers)):
        for i, row in enumerate(neuralNetwork1.layers[layer].weights):
            for j, element in enumerate(row):

                neuralNetwork1.layers[layer].weights[i, j] += epsilon
                output_nn = neuralNetwork1.predict(target_inputs)
                loss_plus = calculate_total_loss(target_output=target_outputs, output_nn=output_nn)

                neuralNetwork1.layers[layer].weights[i, j] -= 2*epsilon
                output_nn = neuralNetwork1.predict(target_inputs)
                loss_minus = calculate_total_loss(target_output=target_outputs, output_nn=output_nn)

                # Approximate the gradient using finite differences
                gradient_approx = (loss_plus - loss_minus) / (2 * epsilon)

              # Check that the gradient is correct
                if abs(gradient_approx + neuralNetwork2.layers[layer].current_delta_weight[i,j]) > 1e-4:
                    is_correct = False

    if is_correct:
        print("Coretto")
    else:
        print("Incorretto")

In [540]:
target_inputs = np.matrix(np.random.rand(100, 2)) # deve essere una matrice
target_outputs = target_inputs*2 # deve essere una matrice

In [541]:
activation_function = IdentityFunction()
layer1 = Layer(num_neurons=3, num_inputs=2, activation_function=activation_function)
layer2 = Layer(num_neurons=4, num_inputs=3, activation_function=activation_function)
layer3 = Layer(num_neurons=2, num_inputs=4, activation_function=activation_function)
neuralNetwork1 = NeuralNetwork([layer1, layer2, layer3])

neuralNetwork2 = NeuralNetwork([copy.deepcopy(layer1), copy.deepcopy(layer2), copy.deepcopy(layer3)])

In [542]:
gradient_check(neuralNetwork1=neuralNetwork1, neuralNetwork2=neuralNetwork2, target_inputs=target_inputs, target_outputs=target_outputs)

Coretto
